<h2>HSTaXe Installation</h2>

<li>conda create --name hstaxe python=3 
<li>conda activate hstaxe
<li>git clone https://github.com/spacetelescope/hstaxe.git
<li>conda install numpy astropy gsl cfitsio wcstools stwcs stsci.imagestats drizzle drizzlepac jupyter
<li>cd hstaxe
<li>python setup.py install

<h2> Load a few Python modules

In [ ]:
from astropy.io import fits
import numpy as np
import os, shutil
import matplotlib.pyplot as plt
%matplotlib inline

from drizzlepac import astrodrizzle

from hstaxe import axetasks




We save the current working directory

In [ ]:
cwd = os.getcwd()
print("We are in %s" % (cwd))

<H2>Preparing the G141 data

We will create a G141 subdirectory to copy all of the G141 files into. This where we will prepare the G141 data

Creating the directory, removing any existing one

In [ ]:
os.chdir(cwd)
if os.path.isdir("G141"):
    shutil.rmtree("G141")
os.mkdir("G141")

Copying the G141 data (which we grab from our cookbook_data directory)

In [ ]:
os.system("cp cookbook_data/G141/*flt.fits G141/")
os.system("cp cookbook_data/G141/G141.lis G141/")

We move into the G141 directory and check the content of the G141.lis file

In [ ]:
os.chdir(cwd)
os.chdir("G141")
!cat G141.lis

We now create a G141 mosaic using the G141 data

This mosaic will be used to set up the proper astrometry for each individual FLT files. We can only extract G141 spectra from FLT files which have been used to make this mosaic

In [ ]:
astrodrizzle.AstroDrizzle("@G141.lis",output="G141",build=True)

We now prepare the F140W Imaging data

We already created a mosaic of all the G141 data for astrometric purposes, and we now create an F140W mosaic using the G141 mosaic as the astrometric reference frame. This will ensure that the G141 and F140W mosaics have pixels with the same RA and DEC. 
The process is similar to what we did with the G141 data and we do this in a F140W sub-directory. The F140W mosaic is generated with the same WCS as the G141 mosaic we already generated.

Creating the directory, removing any existing one

In [ ]:
os.chdir(cwd)

if os.path.isdir("F140W"):
    shutil.rmtree("F140W")

os.mkdir("F140W")


Copy the F140W data (which we grab from our cookbook_data directory)

In [ ]:
os.system("cp cookbook_data/F140W/*flt.fits F140W/")
os.system("cp cookbook_data/F140W/F140W.lis F140W/")

We move into the F140W directory and check the content of the F140W.lis file

In [ ]:
os.chdir(cwd)
os.chdir("F140W")
!cat F140W.lis

We create a F140W mosaic using the F140W data and the G141 mosaic as a reference

In [ ]:
ref = "../G141/G141_drz.fits[1]"
astrodrizzle.AstroDrizzle("@F140W.lis",output="F140W",in_memory=False,skysub="yes",build=True,driz_cr_corr=True,driz_cr=True,final_wcs=True,driz_separate=True,driz_sep_wcs=True,driz_sep_refimage=ref,final_refimage=ref)

The F140W and G141 should be aligned and bright objects should generate bright spectra in the expected position. We should see very liittle offset in the y-direction for WFC3 IR grism data

In [ ]:
plt.rcParams["figure.figsize"] = (10,7)
plt.subplot(1,2,1)
d = fits.open("F140W_drz.fits")[1].data
im1 = plt.imshow(d,origin="lower")
im1.set_clim(0,.2)

plt.subplot(1,2,2)
d = fits.open("../G141/G141_drz.fits")[1].data
im1 = plt.imshow(d,origin="lower")
im1.set_clim(0,.2)

We create an object catalog using sextractor

This is one step that needs to be done carefully as several things can go wrong.
- Make sure you set the magnitude zeropoint properly for the image you are using
- One can generate a simple catalog using:

    sex -c aXe.sex F140W_drz.fits[1] -DETECT_THRESH 5 -MAG_ZEROPOINT 26.4525
    
    
- See aXe.param for the required parameters that aXe will be looking for
- Check the resulting catalog to make sure that all objects have good magnitudes (i.e. no mag of 99.)
- Edit cookbook.cat and rename column MAG_ISO with MAG_F1392, or you will get an "aXeError: Catalogue: test.cat does not contain any magnitude column!" error when running iolprep

This catalog, when doing a simple extraction, will be used to compute the SED of each sources. These SEDs will be used to compute our contamination models. In this example, we used a single band, F140W, but we could have added information in other bands such as F125W for example. This requires running Sextractor in matched photometry mode, and the creation of a catalog where magnitudes in multiple bands are properly listed

For simplicity, here, we copy an already generated catalog:

In [ ]:
os.system("cp ../cookbook_data/cookbook.cat .")
!cat cookbook.cat

We can now run aXe

We start by setting up some necessary environment variables that point to the various aXe directories.

Create a directory called CONF and copy the WFC3 G141 IR Calibration files in there.

In [ ]:
os.chdir(cwd)

if os.path.isdir("CONF"):
    shutil.rmtree("CONF")
os.mkdir("CONF")

os.system("cp cookbook_data/CONF/* CONF/")

Set up some work directories and environment variables required by aXe:

In [ ]:
import os

os.chdir(cwd)

if os.path.isdir("DATA"):
    shutil.rmtree("DATA")
os.mkdir("DATA")
os.environ['AXE_IMAGE_PATH'] = './DATA/' 
print ('--> variable AXE_IMAGE_PATH   set to "./DATA/"')

os.environ['AXE_CONFIG_PATH'] = './CONF/'
print ('--> variable AXE_CONFIG_PATH  set to "./CONF/"')

if os.path.isdir("OUTPUT"):
    shutil.rmtree("OUTPUT")
os.mkdir("OUTPUT")
os.environ['AXE_OUTPUT_PATH'] = './OUTPUT/'
print ('--> variable AXE_OUTPUT_PATH  set to "./OUTPUT/"')

print ("Length of AXE_IMAGE_PATH is",len(os.environ['AXE_IMAGE_PATH']),"characters")


We define the FOV boundaries for the WFC3 IR observations

In [ ]:
dimension_info = "183,85,50,50"

We copy the G141 FLT files and the F140W FLT files in the DATA directory

You can either use the original data or optionally the FLT files used to create the G141 mosaic earlier, which will have some extra bad pixels flagging

In [ ]:
os.system("cp G141/*flt.fits DATA/")
os.system("cp F140W/*flt.fits DATA/")

We use the iolprep aXe task to generate individual F140W catalogs

This task will create the individual F140W extraction catalogs, one for each of the files listed in the F140W.lis file. 
We pass the F140W mosaic to it, as it contains all the information about all the individual F140W FLT file.

In [ ]:
os.chdir(cwd)
os.chdir("F140W")

axetasks.iolprep(drizzle_image='F140W_drz.fits',
                     input_cat='cookbook.cat',
                     dimension_in=dimension_info)

We copy the newly create catalog files into the DATA directory

In [ ]:
os.system("cp ib6o23*_1.cat ../DATA/")

We are almost ready to extract the spectra.
We need to create an file aXe.lis containing the G141 images, expected catalog names and associated F140W direct images

The G141 mosaic we created earlier is not used directly during the aXe extraction process. However, the F140W mosaic was used to create an object master catalog. This catalog will be processed to generate individual object catalogs for the files used to create the F140W mosaic. The aXe.lis file lists which F140W images are logically associated with a particular G141 image. Ideally, these are images taken in the same HST visit so that we can be sure that the WCS of both files are consistent.

The aXe.lis file is a simple text file, with a slightly different format than the one above. In this file, each line contains 3 items:
- The name of a G141 FLT file (e.g. [grism_rootname]_flt.fits
- A catalog name with a name of [direct_rootname]_flt_1.cat
- The name of the direct imaging file [direct_rootname]_flt.fits associated with the G141 data and the catalog. 
    
At this stage, we only have items 1 and 2.

In [ ]:
os.chdir(cwd)
os.system("cp cookbook_data/aXe.lis .")
!cat aXe.lis

We run aXeprep. This task will amongst other things take care of background subtracting the G141 data using a single master sky.

In [ ]:
os.chdir(cwd)

axetasks.axeprep(inlist="aXe.lis",
                     configs="G141.F140W.V4.31.conf",
                     backgr=True,
                     backims="WFC3.IR.G141.sky.V1.0.fits",
                     norm=False,
                     mfwhm=3.0)


We can now proceed with a simple box extraction of our G141 spectra. This will not combine individual 1D spectra and we create one extracted spectrum per object and get G141 FLT file we are processing. 
The contamination is estimated using the Gaussian model of each object that is included in the  SExtractor object catalog test.cat.

For each of the G141 input FLT file, this will create the following in the OUTPUT/ directory:
- [rootname]_flt_2.cat : Object catalog for the FLT file [rootname]_flt.fits
- [rootname]_flt_2.OAF : Aperture file
- [rootname]_flt_2.PET.fits : The Pixel Extraction Table, containing all the unbinned information about each spectrum
- [rootname]_flt_2.SPC.fits : 1D extracted spectra
- [rootname]_flt_2.CONT.fits : Contamination estimate for eact of the spectra
- [rootname]_flt_2_opt.SPC.fits : Optimally extracted version of the 1D spectra

While running the next notebook cell, please check the main terminal window as the aXe routines will output things in there. It will take a few minutes to run


In [ ]:
axetasks.axecore('aXe.lis',
                 "G141.F140W.V4.31.conf",
                 extrfwhm=4.,
                 drzfwhm=3.,
                 backfwhm=0.,
                 orient=False,
                 weights=True,
                 slitless_geom=False,
                 cont_model='gauss',
                 sampling='drizzle',
                 exclude=True)


Results are in the directory pointed to by os.environ['AXE_OUTPUT_PATH'], i.e. ./OUTPUT 
1D and 2D spectra extracted from individual FLT files are available. These are not combined.
SPC files contained 1D spectra, opt.SPC files contained optimally extracted spectra (using gaussian profiles), STP files contain 2D stamps. CONT files contain the contamination estimate (gaussian based)

In [ ]:
!ls OUTPUT/*SPC.fits
!ls OUTPUT/*STP.fits

We can examine individual 2D spectra from the STP files. Note that the STP files are meant for quality control and are not calibrated versions of the 2D spectra. 

In [ ]:
ID = 160

plt.rcParams["figure.figsize"] = (15,3)
plt.subplot(2,2,1)
try:
    d1 = fits.open("OUTPUT/ib6o23rsq_flt_2.STP.fits")["BEAM_%dA" % (ID)].data
    im1 = plt.imshow(d1,origin="lower")
    im1.set_clim(0,.1)
except:
    pass

plt.subplot(2,2,2)
try:
    d1 = fits.open("OUTPUT/ib6o23ryq_flt_2.STP.fits")["BEAM_%dA" % (ID)].data
    im1 = plt.imshow(d1,origin="lower")
    im1.set_clim(0,.1)
except:
    pass

plt.subplot(2,2,3)
try:
    d1 = fits.open("OUTPUT/ib6o23ruq_flt_2.STP.fits")["BEAM_%dA" % (ID)].data
    im1 = plt.imshow(d1,origin="lower")
    im1.set_clim(0,.1)
except:
    pass

plt.subplot(2,2,4)
try:
    d1 = fits.open("OUTPUT/ib6o23s0q_flt_2.STP.fits")["BEAM_%dA" % (ID)].data
    im1 = plt.imshow(d1,origin="lower")
    im1.set_clim(0,.1)
except:
    pass



We now examine the calibrated 1D spectra of one of the sources:

In [ ]:
import glob
for s in glob.glob("OUTPUT/ib6o2*2.SPC.fits"):
    print( s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    f = d1["FLUX"]
    e = d1["FERROR"]
    vg = (w>11500) & (w<16000)
    plt.errorbar(w[vg],f[vg],e[vg])
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux ($erg/s/cm^2/\AA/s$)');

Contamination is not automatically removed but has been estimated and we can plot it

In [ ]:
import glob
for s in glob.glob("OUTPUT/ib6o2*2.SPC.fits"):
    print (s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    c = d1["CONTAM"]
    vg = (w>11500) & (w<16000)
    plt.plot(w[vg],c[vg],label=s)
plt.legend()
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux ($erg/s/cm^2/\AA/s$)');

<h1>More advanced extraction

We now use the aXe Fluxcube extraction

- This method uses multiple mosaics, generated with the same WCS as the G141 mosaic and the SeXtractor segmentation file seg.fits which was generated above when we created the catalog, to generate more accurate contamination estimates and weighted extraction.
- The use of aXedrizzle allows us to combine 2D and 1D spectra taken at the same orientation, which is the case of the data in this example.
- In this example, we use the actual segmenation map to determine the shape of objects. This is done via a set of Fluxcube files was we need to first generate

We create a new directory called DRIZZLE that we will need:

In [ ]:

if os.path.isdir("DRIZZLE"):
    shutil.rmtree("DRIZZLE")
os.mkdir("DRIZZLE")
os.environ['AXE_DRIZZLE_PATH'] = './DRIZZLE/' 
print ('--> variable AXE_DRIZZLE_PATH  set to "./DRIZZLE/"')

We already have an F140W mosaic and we now create an F125W mosaic with the same final WCS as the G141 mosaic

In [ ]:
os.chdir(cwd)

if os.path.isdir("F125W"):
    shutil.rmtree("F125W")

os.mkdir("F125W")


In [ ]:
os.system("cp cookbook_data/F125W/*flt.fits F125W/")
os.system("cp cookbook_data/F125W/F125W.lis F125W/")
os.chdir("F125W")
!cat F125W.lis

In [ ]:
ref = "../G141/G141_drz.fits[1]"
astrodrizzle.AstroDrizzle("@F125W.lis",output="F125W",in_memory=False,skysub="yes",build=True,driz_cr_corr=True,driz_cr=True,final_wcs=True,driz_separate=True,driz_sep_wcs=True,driz_sep_refimage=ref,final_refimage=ref)

We create a directory ./FLX/ to prepare our Fluxcubes:

In [ ]:
os.chdir(cwd)

if os.path.isdir("FLX"):
    shutil.rmtree("FLX")

os.mkdir("FLX")

Copy the direct imaging and G141 mosaic in the Fluxcube directory:

In [ ]:
os.chdir("FLX")

os.system("cp ../F125W/F125W_drz.fits ./")
os.system("cp ../F140W/F140W_drz.fits ./")
os.system("cp ../G141/G141_drz.fits ./")

We will need access to the G141 FLT files, and copy them here instead of working in the ./DATA directory

In [ ]:
os.system("cp ../DATA/*flt.fits .")

We also need the segmenation file we created earlier. Here we copy one we already prepared.

In [ ]:
os.system("cp ../cookbook_data/catalog/seg.fits .")

Create a cubelist.lis file that contains a description of the mosaics we are using and populates the PHOTPLAM and AB magnitude zeropoints

In [ ]:
import glob

dir_images = []
for dir_image in glob.glob("F*drz.fits"):
    print (dir_image)
    fname = dir_image.split("_")[0]
    dir_images.append(dir_image)
        
s = []
for dir_image in dir_images:
    print (dir_image)  
    PHOTPLAM = fits.open(dir_image)[0].header["PHOTPLAM"] # Wavelength of filter in A
    PHOTFLAM = fits.open(dir_image)[0].header["PHOTFLAM"] # Wavelength of filter in A
    ABZP = -48.60 - 2.5*np.log10(PHOTFLAM * PHOTPLAM**2/3e8/1e10 )
    ss = "%s %f %f\n" % (dir_image, PHOTPLAM/10., ABZP)
    s.append(ss)
open("cubelist.lis","w").writelines(s)
    

In [ ]:
!cat cubelist.lis

We now can create the Flexcube

This will combined the mosaics and segmentation file into a flexcube that can be used to compute the contamination and perform the extraction. A different one is generated for each G141 FLT file

In [ ]:
axetasks.fcubeprep(grism_image = os.path.join("G141_drz.fits"),
    segm_image = os.path.join("seg.fits"),
    filter_info = "cubelist.lis",
    AB_zero = "yes",
    dim_info = dimension_info)
os.system("cp ib6o23*FLX.fits ../DATA/")

At this point, we should have generated 2 master FLX file, one for each input imaging mosaic, and four FLX files, one for each of our G141 FLT file.


In [ ]:
plt.rcParams["figure.figsize"] = (10,7)

plt.subplot(1,3,1)
print (np.max(fits.open("ib6o23ryq_flt_2.FLX.fits")[1].data))

im = plt.imshow(fits.open("ib6o23ryq_flt_2.FLX.fits")["SEGM"].data*1.,origin="lower")
im.set_clim(0,199)

plt.subplot(1,3,2)
im = plt.imshow(fits.open("ib6o23ryq_flt_2.FLX.fits")["LAMBDA1248"].data,origin="lower")
im.set_clim(0,1e-20)

plt.subplot(1,3,3)
im = plt.imshow(fits.open("ib6o23ryq_flt_2.FLX.fits")["LAMBDA1392"].data,origin="lower")
im.set_clim(0,1e-20)

We want to work on the non background subtracted G141 data. The ones in the DATA directory have already been subtracted during our basic extraction, so we copy the original G141 data back into the DATA directory.

In [ ]:
os.chdir(cwd)

os.system("cp G141/*flt.fits DATA/")

We run aXeprep on the data. This step also substracts the background.

In [ ]:
!mkdir DRIZZLE/tmp # required for now

In [ ]:
axetasks.axeprep(inlist="aXe.lis",
                     configs="G141.F140W.V4.31.conf",
                     backgr=True,
                     backims="WFC3.IR.G141.sky.V1.0.fits",
                     norm=False,
                     mfwhm=3.0)

Checking the background levels that were subtracted from each fo the G141 FLT files

In [ ]:
print( fits.open("DATA/ib6o23rsq_flt.fits")[1].header["SKY_CPS"],"e/s")
print( fits.open("DATA/ib6o23ruq_flt.fits")[1].header["SKY_CPS"],"e/s")
print( fits.open("DATA/ib6o23ryq_flt.fits")[1].header["SKY_CPS"],"e/s")
print( fits.open("DATA/ib6o23s0q_flt.fits")[1].header["SKY_CPS"],"e/s")

We run axecore using the flexcube models

In [ ]:
axetasks.axecore('aXe.lis',
                 "G141.F140W.V4.31.conf",
                 extrfwhm=4.,
                 drzfwhm=3.,
                 backfwhm=0.,
                 orient=False,
                 weights=True,
                 slitless_geom=False,
                 cont_model='fluxcube',
                 sampling='drizzle',
                 exclude=True)

Now we use aXedrizzle to combine the dithered observations into single 2D and 1D spectra.
We first run the drzprep routine and then the axedrizzle task. The latter will take several minutes to run.

<h2> Do a basic box extraction

In [ ]:
opt_extr=False

In [ ]:
axetasks.drzprep(inlist = "aXe.lis",
            configs =  "G141.F140W.V4.31.conf",
            back = False,opt_extr=opt_extr)

In [ ]:
axetasks.axecrr(inlist="aXe.lis",
    configs="G141.F140W.V4.31.conf",
    infwhm = 4.0,
    outfwhm = 3.0,
    back = False,
    driz_separate = 'yes',
    opt_extr=opt_extr
    )


<h2> Do an Optimal extraction

In [ ]:
opt_extr=True

In [ ]:
axetasks.drzprep(inlist = "aXe.lis",
            configs =  "G141.F140W.V4.31.conf",
            back = False,opt_extr=opt_extr)

In [ ]:
axetasks.axecrr(inlist="aXe.lis",
    configs="G141.F140W.V4.31.conf",
    infwhm = 4.0,
    outfwhm = 3.0,
    back = False,
    driz_separate = 'yes',
    opt_extr=opt_extr
    )

The extraction results are in the DRIZZLE directory we created, and we can examine a 2D, rectified and wavelength calibrated version of the spectrum we looked at earlier:

In [ ]:
ID = 160
d = fits.open("./DRIZZLE/aXeWFC3_G141_2.STP.fits")["BEAM_%dA" % (ID)].data
im = plt.imshow(d)
im.set_clim(0,0.1)

We plot the extracted 1D spectra of our source and the estimate of the contamination:

In [ ]:
fin = fits.open("./DRIZZLE/aXeWFC3_G141_2.SPC.fits")
tdata = fin["BEAM_%dA" % (ID)].data
x = tdata["LAMBDA"]
f = tdata["FLUX"]
e = tdata["FERROR"]

c = tdata["CONTAM"]
vg = (x>11500) & (x<16500)
plt.plot(x[vg],f[vg])
plt.errorbar(x[vg],f[vg],e[vg])

plt.plot(x[vg],c[vg])

The MEF files in the DRIZZLE directory contain the 2D version of the spectrum of a source as well as estimte of the contamination:

In [ ]:
plt.subplot(3,1,1)
d = fits.open("./DRIZZLE/aXeWFC3_G141_mef_ID%d.fits" % (ID))["SCI"].data
im = plt.imshow(d,origin="lower")
im.set_clim(0,0.05)

plt.subplot(3,1,2)
d = fits.open("./DRIZZLE/aXeWFC3_G141_mef_ID%d.fits" % (ID))["CON"].data
im = plt.imshow(d,origin="lower")
im.set_clim(0,0.05)

The individually extracted spectra are in the OUTPUT directory and the combined ones in the DRIZZLE directory. We can plot and compare them:

In [ ]:
import glob

for s in glob.glob("OUTPUT/ib6o2*2.SPC.fits"):
    print (s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    f = d1["FLUX"]
    e = d1["FERROR"]
    vg = (w>11500) & (w<16000)
    plt.errorbar(w[vg],f[vg],e[vg])
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux ($erg/s/cm^2/\AA/s$)');


fin = fits.open("./DRIZZLE/aXeWFC3_G141_2.SPC.fits")
tdata = fin["BEAM_%dA" % (ID)].data
x = tdata["LAMBDA"]
f = tdata["FLUX"]
e = tdata["FERROR"]

c = tdata["CONTAM"]
vg = (x>11500) & (x<16500)
#plt.errorbar(x[vg],y[vg],e[vg])
plt.plot(x[vg],f[vg],color='k',lw=2)
plt.errorbar(x[vg],f[vg],e[vg],color='k',lw=2)
